In [1]:
from dotenv import load_dotenv
import sys

sys.path.append("../common")
load_dotenv()

True

In [2]:
import os
from langsmith_tracker import set_tracking
from langchain_print import stream_response

# 인스턴스를 생성할 때 필요한 매개변수를 전달합니다.
set_tracking(project_name="03.OutputParser")

Langsmith 추적이 활성화되었습니다. [프로젝트명: 03.OutputParser]


### PandasdataFrameOutputParser

`Pandas DataFrame` 은 Python 언어에서 널리 사용되는 데이터 구조입니다.  
데이터 조작 및 분석을 위한 도구입니다. 데이터의 정젝, 변환 및 분석과 같은 다양한 작업을 수행할 수 있습니다.   
`Python` 을 한다면 꼭 스킬셋에 장착되어 있어야 하는 기술입니다.  
  
이 출력파서는 사용자가 임의의 Pandas DataFrame 을 지정하여 해당 DataFrame 에서 데이터를 추출하고, 이를 혁싱화된 dict 형태로 조회할 수 있게 해주는 LLM 기반의 도구입니다.

In [3]:
import pprint  # 이쁘게 출력
from typing import Any, Dict

import pandas as pd
from langchain.output_parsers import PandasDataFrameOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [29]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

`format_parser_output` 함수는 출력 파서를 dict 형식으로 변환하고 출력형식을 지정하는데 하용합니다.

In [20]:
# dict로 변환, 이쁘게 출력
def format_parser_output(output_parser: Dict[str, Any]) -> None:
    # output_parser fm를
    for key in output_parser.keys():
        output_parser[key] = output_parser[key].to_dict()
    return pprint.PrettyPrinter(width=4, compact=True).pprint(output_parser)

csv 파일을 불러옵니다.

In [21]:
df = pd.read_csv("./data/NVIDIA_latest_graphics_cards.csv")
df.head()

,Model,Release Date,Architecture,Memory,Key Features
0,GeForce RTX 4090,2022-10,Ada Lovelace,24GB G6X,"High performance gaming, AI-powered graphics, ..."
1,GeForce RTX 4080 SUPER,2024-01,Ada Lovelace,16GB G6X,"Enhanced compute speed, improved AI applicatio..."
2,GeForce RTX 4070 Ti SUPER,2024-01,Ada Lovelace,12GB G6X,Improved performance and competitive pricing
3,GeForce RTX 4070 SUPER,2024-01,Ada Lovelace,12GB G6X,Improved efficiency and performance
4,GeForce RTX 4060 Ti,2023-05,Ada Lovelace,8GB G6,"DLSS 3 support, power-efficient consumption"


In [32]:
output_parser = PandasDataFrameOutputParser(dataframe=df)

print(output_parser.get_format_instructions())

The output should be formatted as a string as the operation, followed by a colon, followed by the column or row to be queried on, followed by optional array parameters.
1. The column names are limited to the possible columns below.
2. Arrays must either be a comma-separated list of numbers formatted as [1,3,5], or it must be in range of numbers formatted as [0..4].
3. Remember that arrays are optional and not necessarily required.
4. If the column is not in the possible columns or the operation is not a valid Pandas DataFrame operation, return why it is invalid as a sentence starting with either "Invalid column" or "Invalid operation".

As an example, for the formats:
1. String "column:num_legs" is a well-formatted instance which gets the column num_legs, where num_legs is a possible column.
2. String "row:1" is a well-formatted instance which gets row 1.
3. String "column:num_legs[1,2]" is a well-formatted instance which gets the column num_legs for rows 1 and 2, where num_legs is a p

컬럼에 대해 조회할 수 있습니다.

In [35]:
#  열 작업 예시
question = "Memory 컬럼을 조회해주세요."

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{question}\n",
    input_variables=["question"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

chain = prompt | llm | output_parser

response = chain.invoke({"question", question})

format_parser_output(response)

{'Memory': {0: '24GB '
               'G6X',
            1: '16GB '
               'G6X',
            2: '12GB '
               'G6X',
            3: '12GB '
               'G6X',
            4: '8GB '
               'G6'}}
